In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import os
import sys
sys.path.append(os.path.abspath('../'))
from simulation import model
from simulation.data import io
from simulation import simulation

In [2]:
s = pd.Series([v for k, v in dict(model.bipartite_graph.degree()).items() if k in model.channels])

In [4]:
'{:.2%}'.format(s[s>2].size / s.size)

'33.99%'

In [ ]:
ACMBlue = '#00cfe6'
ACMYellow = '#ffd600'
ACMOrange = '#fc9200'
ACMRed = '#ff1924'
ACMLightBlue = '#82fcff'
ACMGreen = '#a5cf00'
ACMPurple = '#6200d9'
ACMDarkBlue = '#0055c9'

plt.rcParams.update({
    "font.family": "serif",  # use serif/main font for text elements
    "text.usetex": True,     # use inline math for ticks
    "pgf.rcfonts": False     # don't setup fonts from rc parameters
    })

# Time-ignoring horizon

In [ ]:
time_ignoring_horizon = io.load_horizon_cardinalities(is_time_respecting=False, file_name='simulation_results.hdf')

In [ ]:
time_ignoring_horizon.describe()

Largest components

In [ ]:
for s in sorted([len(cc & model.participants) for cc in nx.connected_components(model.bipartite_graph)], reverse=True)[0:5]:
    print(s, '{:.2%}'.format(s/len(model.participants)))

# Time-respecting horizon

In [ ]:
time_respecting_horizon = io.load_simulation_results(time_respecting=True)

In [ ]:
time_respecting_horizon.describe()

In [ ]:
'{:.2%}'.format(26216/len(model.participants))

# Difference between both models

In [ ]:
diff = time_ignoring_horizon - time_respecting_horizon

In [ ]:
diff.describe()

# Plots

In [ ]:
fig, ax = plt.subplots(figsize=(7.22, 7.22/3), dpi=350, constrained_layout=True)

violinplot = ax.violinplot([time_respecting_horizon, time_ignoring_horizon], vert=False, showmeans=True, showmedians=True, points=1000, widths=1, showextrema=True);

for i in violinplot['bodies']:
    i.set_facecolor('black')

violinplot['cmeans'].set_color(ACMDarkBlue)
violinplot['cmeans'].set_label('Mean')

violinplot['cmedians'].set_color(ACMRed)
violinplot['cmedians'].set_label('Median')

for k in ['cmaxes', 'cmins', 'cbars']:
    violinplot[k].set_color('gray')

yticklabels = ['Time-ignoring', 'Time-respecting']
ax.set_yticks([1, 2])
ax.set_yticklabels(yticklabels)
ax.set_xlabel('Cardinality of horizon')
ax.xaxis.set_major_formatter(plt.matplotlib.ticker.StrMethodFormatter('{x:,.0f}'))

ax.grid(which='both', axis='x', linestyle='--')
plt.legend(loc='upper center');

plt.savefig('../plots/dist.pgf', dpi=1000, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(3.5, 3.5), dpi=150, constrained_layout=True);

violinplot = ax.violinplot(diff, vert=True, showmeans=True, showmedians=True, points=1000, widths=0.8, showextrema=True);

for i in violinplot['bodies']:
    i.set_facecolor('black')

violinplot['cmeans'].set_color(ACMDarkBlue)
violinplot['cmeans'].set_label('Mean')

violinplot['cmedians'].set_color(ACMRed)
violinplot['cmedians'].set_label('Median')

for k in ['cmaxes', 'cmins', 'cbars']:
    violinplot[k].set_color('gray')

ax.set_xticks([], minor=False)
ax.set_ylabel('Cardinality of horizon')
ax.yaxis.set_major_formatter(plt.matplotlib.ticker.StrMethodFormatter('{x:,.0f}'))

ax.grid(which='both', axis='y', linestyle='--')
plt.legend(loc='best');
plt.savefig('../plots/diff.pgf', dpi=1000, bbox_inches = 'tight')
